# PointNet Training & Verification with α,β-CROWN

## LiDAR Point Cloud Classification + Formal Verification

This notebook:
1. **Loads** your LiDAR data from Google Drive
2. **Trains** PointNet (original architecture) on GPU
3. **Verifies** robustness properties using α,β-CROWN API

### Architecture: Original PointNet (Qi et al., CVPR 2017)
- Input T-Net (3x3) for spatial alignment
- Feature T-Net (64x64) for feature alignment  
- Point-wise MLP: 7→64→64→64→128→1024 (5 conv layers with BatchNorm)
- Global max pooling
- Classifier MLP: 1024→512→256→2 (with BatchNorm + Dropout)
- **~3.5M parameters**

### Input Format: (N, 1024, 7)
- xyz coordinates (3 channels)
- Geometric features (4 channels): linearity, curvature, density_var, planarity

### Properties Verified:

**Property 1: Local Robustness (L∞)**
```
∀x' with ||x' - x₀||∞ ≤ ε : f(x') = f(x₀)
```

**Property 2: Safety Property**
```
∀x' with ||x' - x₀||∞ ≤ ε ∧ f(x₀)=CRITICAL : f(x') ≠ NON_CRITICAL
```

## 1. Setup and Installation

In [11]:
# Check GPU
!nvidia-smi

Fri Jan  2 17:52:23 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# Install dependencies
!pip install torch numpy onnx onnxruntime pyyaml packaging appdirs sortedcontainers -q

# Clone alpha-beta-CROWN
!git clone https://github.com/Verified-Intelligence/alpha-beta-CROWN.git 2>/dev/null || true

# NOTE: Google Drive mount removed - using local storage instead
# If you need to upload data, use the file browser on the left or:
# from google.colab import files
# uploaded = files.upload()  # Upload your .npy files

In [ ]:
# Setup paths and imports
import sys
sys.path.insert(0, '/content/alpha-beta-CROWN/auto_LiRPA')
sys.path.insert(0, '/content/alpha-beta-CROWN/complete_verifier')

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import json
from datetime import datetime
from torch.utils.data import TensorDataset, DataLoader

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
print(f"Device: {device}")

In [ ]:
# Mock onnxoptimizer to avoid compilation issues
from types import ModuleType
mock = ModuleType('onnxoptimizer')
mock.optimize = lambda model, passes=None: model
sys.modules['onnxoptimizer'] = mock

# Import auto_LiRPA for verification
from auto_LiRPA import BoundedModule, BoundedTensor, PerturbationLpNorm
print("auto_LiRPA imported successfully!")

In [ ]:
# Set random seeds for reproducibility
import random
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

In [ ]:
# Configuration
N_POINTS = 1024        # Points per sample (original PointNet)
IN_CHANNELS = 7        # xyz(3) + features(4)
NUM_CLASSES = 2        # CRITICAL vs NON_CRITICAL
INPUT_DIM = N_POINTS * IN_CHANNELS  # 7168

# Training config
EPOCHS = 50
BATCH_SIZE = 32
LEARNING_RATE = 0.001

# Verification config
EPSILONS = [0.001, 0.003, 0.005, 0.007, 0.01]  # L-inf perturbation budgets
N_VERIFY_SAMPLES = 20  # Samples to verify

print(f"Configuration:")
print(f"  Points per sample: {N_POINTS}")
print(f"  Input channels: {IN_CHANNELS} (xyz + 4 geometric features)")
print(f"  Input dimension: {INPUT_DIM}")
print(f"  Classes: {NUM_CLASSES}")

## 2. Load Data from GitHub

Data files are stored in the repository using Git LFS.

In [ ]:
# Clone repository with Git LFS data
import os

REPO_URL = "https://github.com/francescacraievich/mola-pointnet-verification.git"
REPO_DIR = "/content/mola-pointnet-verification"
DATA_PATH = f"{REPO_DIR}/data/pointnet"

if not os.path.exists(REPO_DIR):
    print("Cloning repository with Git LFS...")
    !git lfs install
    !git clone {REPO_URL} {REPO_DIR}
    print("Done!")
else:
    print(f"Repository already cloned at {REPO_DIR}")

# Verify data files
data_files = ['train_groups.npy', 'train_labels.npy', 'test_groups.npy', 'test_labels.npy']
for f in data_files:
    path = os.path.join(DATA_PATH, f)
    if os.path.exists(path):
        size = os.path.getsize(path) / 1e6
        print(f"✓ {f}: {size:.1f} MB")
    else:
        print(f"✗ {f}: NOT FOUND")

In [ ]:
# Load the data
train_groups = np.load(os.path.join(DATA_PATH, 'train_groups.npy'))
train_labels = np.load(os.path.join(DATA_PATH, 'train_labels.npy'))
test_groups = np.load(os.path.join(DATA_PATH, 'test_groups.npy'))
test_labels = np.load(os.path.join(DATA_PATH, 'test_labels.npy'))

print(f"Train data: {train_groups.shape}, labels: {train_labels.shape}")
print(f"Test data: {test_groups.shape}, labels: {test_labels.shape}")
print(f"Label distribution (train): CRITICAL={sum(train_labels==0)}, NON_CRITICAL={sum(train_labels==1)}")
print(f"Label distribution (test): CRITICAL={sum(test_labels==0)}, NON_CRITICAL={sum(test_labels==1)}")

# Verify shape
assert train_groups.shape[1] == N_POINTS, f"Expected {N_POINTS} points, got {train_groups.shape[1]}"
assert train_groups.shape[2] == IN_CHANNELS, f"Expected {IN_CHANNELS} channels, got {train_groups.shape[2]}"

In [ ]:
# Create DataLoaders
train_dataset = TensorDataset(
    torch.from_numpy(train_groups).float(),
    torch.from_numpy(train_labels).long()
)
test_dataset = TensorDataset(
    torch.from_numpy(test_groups).float(),
    torch.from_numpy(test_labels).long()
)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

print(f"Train batches: {len(train_loader)}")
print(f"Test batches: {len(test_loader)}")

## 3. PointNet Model (Original Architecture)

Based on Qi et al., "PointNet: Deep Learning on Point Sets" (CVPR 2017)

Architecture:
- **T-Net 3x3**: Spatial transformer for xyz coordinates
- **T-Net 64x64**: Feature transformer after first MLP
- **Point MLP**: 7→64→64→[feat_trans]→64→128→1024 with BatchNorm
- **Global MaxPool**: Symmetric aggregation
- **Classifier**: 1024→512→256→2 with BatchNorm + Dropout(0.3)

In [ ]:
class TNet(nn.Module):
    """
    T-Net: Spatial Transformer Network for PointNet.
    Predicts a k x k transformation matrix.
    
    Architecture (original):
    - Conv1d: k → 64 → 128 → 1024
    - MaxPool
    - FC: 1024 → 512 → 256 → k*k
    - All with BatchNorm
    """
    def __init__(self, k=3):
        super().__init__()
        self.k = k
        
        # Shared MLP (implemented as Conv1d)
        self.conv1 = nn.Conv1d(k, 64, 1)
        self.conv2 = nn.Conv1d(64, 128, 1)
        self.conv3 = nn.Conv1d(128, 1024, 1)
        
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        
        # FC layers
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, k * k)
        
        self.bn_fc1 = nn.BatchNorm1d(512)
        self.bn_fc2 = nn.BatchNorm1d(256)
        
        # Initialize to identity
        self.fc3.weight.data.zero_()
        self.fc3.bias.data.copy_(torch.eye(k).view(-1))
    
    def forward(self, x):
        """
        Args:
            x: (batch, k, n_points)
        Returns:
            transform: (batch, k, k) transformation matrix
        """
        batch_size = x.shape[0]
        
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        
        # Max pool over points
        x = torch.max(x, dim=2)[0]  # (batch, 1024)
        
        x = F.relu(self.bn_fc1(self.fc1(x)))
        x = F.relu(self.bn_fc2(self.fc2(x)))
        x = self.fc3(x)
        
        # Reshape to k x k matrix
        x = x.view(batch_size, self.k, self.k)
        
        return x


class PointNetForVerification(nn.Module):
    """
    PointNet for Verification - IDENTICAL to original PointNet architecture.
    
    Based on: Qi et al., "PointNet: Deep Learning on Point Sets" (CVPR 2017)
    
    Features:
    - Input T-Net (3x3) only transforms xyz, not extra features
    - Feature T-Net (64x64) after conv2
    - 5 conv layers: in_channels→64→64→64→128→1024
    - BatchNorm on all layers
    - Dropout(0.3) in classifier
    
    Input format: (batch, n_points, in_channels)
    - in_channels=7: xyz(3) + features(4)
    """
    def __init__(
        self,
        num_points=1024,
        num_classes=2,
        use_tnet=True,
        feature_transform=True,
        in_channels=7,
    ):
        super().__init__()
        
        self.num_points = num_points
        self.num_classes = num_classes
        self.use_tnet = use_tnet
        self.feature_transform = feature_transform
        self.in_channels = in_channels
        self.input_dim = num_points * in_channels
        
        # Input T-Net (3x3) - only for xyz
        if use_tnet:
            self.input_tnet = TNet(k=3)
        
        # Feature T-Net (64x64)
        if feature_transform:
            self.feat_tnet = TNet(k=64)
        
        # Point-wise MLP (5 conv layers)
        self.conv1 = nn.Conv1d(in_channels, 64, 1)
        self.conv2 = nn.Conv1d(64, 64, 1)
        self.conv3 = nn.Conv1d(64, 64, 1)
        self.conv4 = nn.Conv1d(64, 128, 1)
        self.conv5 = nn.Conv1d(128, 1024, 1)
        
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(64)
        self.bn3 = nn.BatchNorm1d(64)
        self.bn4 = nn.BatchNorm1d(128)
        self.bn5 = nn.BatchNorm1d(1024)
        
        # Classifier MLP
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, num_classes)
        
        self.bn_fc1 = nn.BatchNorm1d(512)
        self.bn_fc2 = nn.BatchNorm1d(256)
        self.dropout = nn.Dropout(p=0.3)
    
    def forward(self, x):
        batch_size = x.shape[0]
        
        # Handle flattened input
        if x.dim() == 2:
            x = x.view(batch_size, self.num_points, self.in_channels)
        
        # Separate xyz and extra features
        if self.in_channels > 3:
            xyz = x[:, :, :3]  # (batch, n_points, 3)
            extra_features = x[:, :, 3:]  # (batch, n_points, in_channels-3)
        else:
            xyz = x
            extra_features = None
        
        # Input T-Net (only on xyz)
        if self.use_tnet:
            xyz_t = xyz.transpose(1, 2)  # (batch, 3, n_points)
            input_trans = self.input_tnet(xyz_t)  # (batch, 3, 3)
            xyz = torch.bmm(xyz, input_trans)  # (batch, n_points, 3)
        
        # Recombine
        if extra_features is not None:
            x = torch.cat([xyz, extra_features], dim=2)
        else:
            x = xyz
        
        # Point-wise MLP
        x = x.transpose(1, 2)  # (batch, in_channels, n_points)
        x = F.relu(self.bn1(self.conv1(x)))  # (batch, 64, n_points)
        x = F.relu(self.bn2(self.conv2(x)))  # (batch, 64, n_points)
        
        # Feature T-Net
        if self.feature_transform:
            feat_trans = self.feat_tnet(x)  # (batch, 64, 64)
            x = x.transpose(1, 2)  # (batch, n_points, 64)
            x = torch.bmm(x, feat_trans)  # (batch, n_points, 64)
            x = x.transpose(1, 2)  # (batch, 64, n_points)
        
        x = F.relu(self.bn3(self.conv3(x)))  # (batch, 64, n_points)
        x = F.relu(self.bn4(self.conv4(x)))  # (batch, 128, n_points)
        x = F.relu(self.bn5(self.conv5(x)))  # (batch, 1024, n_points)
        
        # Global max pooling
        x = torch.max(x, dim=2)[0]  # (batch, 1024)
        
        # Classifier
        x = F.relu(self.bn_fc1(self.fc1(x)))
        x = self.dropout(x)
        x = F.relu(self.bn_fc2(self.fc2(x)))
        x = self.dropout(x)
        x = self.fc3(x)
        
        return x
    
    def get_transforms(self, x):
        """Return input and feature transforms for regularization."""
        batch_size = x.shape[0]
        
        if x.dim() == 2:
            x = x.view(batch_size, self.num_points, self.in_channels)
        
        if self.in_channels > 3:
            xyz = x[:, :, :3]
        else:
            xyz = x
        
        input_trans = None
        feat_trans = None
        
        if self.use_tnet:
            xyz_t = xyz.transpose(1, 2)
            input_trans = self.input_tnet(xyz_t)
            xyz = torch.bmm(xyz, input_trans)
        
        if self.in_channels > 3:
            x = torch.cat([xyz, x[:, :, 3:]], dim=2)
        else:
            x = xyz
        
        x = x.transpose(1, 2)
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        
        if self.feature_transform:
            feat_trans = self.feat_tnet(x)
        
        return input_trans, feat_trans


# Create model
model = PointNetForVerification(
    num_points=N_POINTS,
    num_classes=NUM_CLASSES,
    use_tnet=True,
    feature_transform=True,
    in_channels=IN_CHANNELS,
).to(device)

n_params = sum(p.numel() for p in model.parameters())
print(f"PointNet parameters: {n_params:,}")
print(f"Expected: ~3.5M parameters")

## 4. Training

In [ ]:
def feature_transform_regularizer(trans):
    """Regularization loss for feature transform to be close to orthogonal."""
    d = trans.size()[1]
    I = torch.eye(d, device=trans.device).unsqueeze(0)
    loss = torch.mean(torch.norm(I - torch.bmm(trans, trans.transpose(2, 1)), dim=(1, 2)))
    return loss


def train_epoch(model, loader, criterion, optimizer):
    """Train for one epoch."""
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for batch_data, batch_labels in loader:
        batch_data = batch_data.to(device)
        batch_labels = batch_labels.to(device)
        
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(batch_data)
        loss = criterion(outputs, batch_labels)
        
        # Add feature transform regularization
        if model.feature_transform:
            _, feat_trans = model.get_transforms(batch_data)
            if feat_trans is not None:
                loss = loss + 0.001 * feature_transform_regularizer(feat_trans)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item() * batch_data.size(0)
        _, predicted = outputs.max(1)
        correct += predicted.eq(batch_labels).sum().item()
        total += batch_data.size(0)
    
    return total_loss / total, 100.0 * correct / total


def evaluate(model, loader):
    """Evaluate model on dataset."""
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch_data, batch_labels in loader:
            batch_data = batch_data.to(device)
            batch_labels = batch_labels.to(device)
            
            outputs = model(batch_data)
            _, predicted = outputs.max(1)
            correct += predicted.eq(batch_labels).sum().item()
            total += batch_data.size(0)
    
    return 100.0 * correct / total

In [ ]:
# Training loop
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

best_acc = 0
history = {'train_loss': [], 'train_acc': [], 'test_acc': []}

print("="*60)
print("Training PointNet")
print("="*60)

for epoch in range(EPOCHS):
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer)
    test_acc = evaluate(model, test_loader)
    scheduler.step()
    
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['test_acc'].append(test_acc)
    
    if test_acc > best_acc:
        best_acc = test_acc
        # Save best model
        torch.save({
            'model_state_dict': model.state_dict(),
            'n_points': N_POINTS,
            'num_classes': NUM_CLASSES,
            'in_channels': IN_CHANNELS,
            'use_tnet': True,
            'feature_transform': True,
            'test_accuracy': best_acc,
        }, 'pointnet_best.pth')
    
    if (epoch + 1) % 5 == 0 or epoch == 0:
        print(f"Epoch {epoch+1:3d}/{EPOCHS} | Loss: {train_loss:.4f} | "
              f"Train Acc: {train_acc:.1f}% | Test Acc: {test_acc:.1f}% | Best: {best_acc:.1f}%")

print(f"\nTraining complete! Best accuracy: {best_acc:.2f}%")

In [ ]:
# Plot training curves
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Loss
axes[0].plot(history['train_loss'], 'b-', label='Train Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Accuracy
axes[1].plot(history['train_acc'], 'b-', label='Train Acc')
axes[1].plot(history['test_acc'], 'r-', label='Test Acc')
axes[1].axhline(y=best_acc, color='g', linestyle='--', label=f'Best: {best_acc:.1f}%')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy (%)')
axes[1].set_title('Training & Test Accuracy')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_curves.png', dpi=150)
plt.show()

In [ ]:
# Load best model for verification
checkpoint = torch.load('pointnet_best.pth', map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()
print(f"Loaded best model with test accuracy: {checkpoint['test_accuracy']:.2f}%")

## 5. Verification with auto_LiRPA

Using auto_LiRPA API directly (native PyTorch support for Conv1d, MaxPool, etc.)

In [ ]:
def verify_robustness_lirpa(model, sample, label, epsilon, method='IBP+backward'):
    """
    Verify local robustness using auto_LiRPA.
    
    Property: ∀x' with ||x' - x₀||∞ ≤ ε : f(x') = f(x₀)
    
    Args:
        model: PyTorch model
        sample: Input sample (n_points, in_channels)
        label: True label
        epsilon: L-inf perturbation bound
        method: Bound propagation method ('IBP', 'IBP+backward', 'CROWN')
    
    Returns:
        dict with verification result
    """
    model.eval()
    model_cpu = model.cpu()
    
    # Prepare input
    sample_tensor = torch.FloatTensor(sample).unsqueeze(0)  # (1, n_points, in_channels)
    
    # Create bounded model
    bounded_model = BoundedModule(model_cpu, sample_tensor, device='cpu')
    
    # Define perturbation
    ptb = PerturbationLpNorm(norm=float('inf'), eps=epsilon)
    bounded_input = BoundedTensor(sample_tensor, ptb)
    
    # Compute bounds
    lb, ub = bounded_model.compute_bounds(x=(bounded_input,), method=method)
    
    # Check if correct class is always highest
    # For binary classification: label=0 means class 0 > class 1, label=1 means class 1 > class 0
    if label == 0:
        # Need lb[0] > ub[1] (class 0 always higher than class 1)
        margin = lb[0, 0] - ub[0, 1]
    else:
        # Need lb[1] > ub[0] (class 1 always higher than class 0)
        margin = lb[0, 1] - ub[0, 0]
    
    verified = margin.item() > 0
    
    # Move model back to device
    model.to(device)
    
    return {
        'verified': verified,
        'margin': margin.item(),
        'lb': lb.detach().numpy(),
        'ub': ub.detach().numpy(),
        'method': method
    }


def verify_safety_lirpa(model, sample, epsilon, method='IBP+backward'):
    """
    Verify safety property using auto_LiRPA.
    
    Property: For CRITICAL samples, no perturbation causes NON_CRITICAL classification.
    
    Args:
        model: PyTorch model
        sample: Input sample (n_points, in_channels)
        epsilon: L-inf perturbation bound
        method: Bound propagation method
    
    Returns:
        dict with verification result
    """
    model.eval()
    model_cpu = model.cpu()
    
    # Prepare input
    sample_tensor = torch.FloatTensor(sample).unsqueeze(0)
    
    # First check prediction
    with torch.no_grad():
        output = model_cpu(sample_tensor)
        pred = output.argmax(dim=1).item()
        confidence = torch.softmax(output, dim=1)[0]
    
    # Only verify if predicted as CRITICAL (class 0)
    if pred != 0:
        model.to(device)
        return {
            'verified': False,
            'status': 'skipped_wrong_prediction',
            'original_prediction': pred,
            'confidence': confidence.numpy()
        }
    
    # Create bounded model
    bounded_model = BoundedModule(model_cpu, sample_tensor, device='cpu')
    
    # Define perturbation
    ptb = PerturbationLpNorm(norm=float('inf'), eps=epsilon)
    bounded_input = BoundedTensor(sample_tensor, ptb)
    
    # Compute bounds
    lb, ub = bounded_model.compute_bounds(x=(bounded_input,), method=method)
    
    # Safety: CRITICAL (class 0) should always have higher score than NON_CRITICAL (class 1)
    # Need lb[0] > ub[1]
    margin = lb[0, 0] - ub[0, 1]
    verified = margin.item() > 0
    
    model.to(device)
    
    return {
        'verified': verified,
        'margin': margin.item(),
        'lb': lb.detach().numpy(),
        'ub': ub.detach().numpy(),
        'method': method,
        'original_prediction': pred,
        'confidence': confidence.numpy()
    }


print("Verification functions defined using auto_LiRPA API!")

In [ ]:
# Property 1: Local Robustness Verification
print("="*70)
print("PROPERTY 1: LOCAL ROBUSTNESS (L∞)")
print("Verifying: ∀x' with ||x' - x₀||∞ ≤ ε : f(x') = f(x₀)")
print("="*70)

robustness_results = {}

for eps in EPSILONS:
    print(f"\nε = {eps}")
    print("-"*40)
    
    verified_count = 0
    total = 0
    
    for i in range(N_VERIFY_SAMPLES):
        sample = test_groups[i]
        label = int(test_labels[i])
        label_str = "CRITICAL" if label == 0 else "NON_CRITICAL"
        
        try:
            result = verify_robustness_lirpa(model, sample, label, eps)
            if result['verified']:
                verified_count += 1
                status = f"✓ VERIFIED (margin={result['margin']:.4f})"
            else:
                status = f"✗ NOT VERIFIED (margin={result['margin']:.4f})"
            total += 1
        except Exception as e:
            status = f"⚠ ERROR: {str(e)[:30]}"
        
        print(f"  Sample {i:3d} ({label_str:12}): {status}")
    
    robustness_results[str(eps)] = {
        'epsilon': eps,
        'verified': verified_count,
        'total': total,
        'verified_pct': 100 * verified_count / total if total > 0 else 0
    }
    
    print(f"\n  Summary: {verified_count}/{total} verified ({robustness_results[str(eps)]['verified_pct']:.1f}%)")

In [ ]:
# Property 2: Safety Verification
print("="*70)
print("PROPERTY 2: SAFETY PROPERTY")
print("Verifying: For CRITICAL samples, never misclassified as NON_CRITICAL")
print("="*70)

# Get CRITICAL samples (label=0)
critical_indices = np.where(test_labels == 0)[0]
n_critical = min(N_VERIFY_SAMPLES, len(critical_indices))

safety_results = {}

for eps in EPSILONS:
    print(f"\nε = {eps}")
    print("-"*40)
    
    verified_count = 0
    skipped_count = 0
    total = 0
    
    for i, idx in enumerate(critical_indices[:n_critical]):
        sample = test_groups[idx]
        
        try:
            result = verify_safety_lirpa(model, sample, eps)
            
            if result.get('status') == 'skipped_wrong_prediction':
                skipped_count += 1
                status = f"⊘ SKIPPED (model predicts NON_CRITICAL)"
            elif result['verified']:
                verified_count += 1
                status = f"✓ SAFE (margin={result['margin']:.4f})"
                total += 1
            else:
                status = f"✗ UNSAFE (margin={result['margin']:.4f})"
                total += 1
        except Exception as e:
            status = f"⚠ ERROR: {str(e)[:30]}"
        
        print(f"  Sample {idx:3d} (CRITICAL): {status}")
    
    safety_results[str(eps)] = {
        'epsilon': eps,
        'verified': verified_count,
        'total': total,
        'skipped': skipped_count,
        'verified_pct': 100 * verified_count / total if total > 0 else 0
    }
    
    print(f"\n  Summary: {verified_count}/{total} safe ({safety_results[str(eps)]['verified_pct']:.1f}%), {skipped_count} skipped")

## 6. Results Summary

In [ ]:
# Results Summary
print("\n" + "="*70)
print("FINAL RESULTS SUMMARY")
print("="*70)

# Property 1 Table
print("\n### PROPERTY 1: LOCAL ROBUSTNESS ###")
print(f"{'Epsilon':>10} | {'Verified':>10} | {'Total':>10} | {'Verified %':>12}")
print("-"*50)
for eps_str, r in robustness_results.items():
    print(f"{float(eps_str):>10.4f} | {r['verified']:>10} | {r['total']:>10} | {r['verified_pct']:>10.1f}%")

# Property 2 Table
print(f"\n### PROPERTY 2: SAFETY (CRITICAL -> never NON_CRITICAL) ###")
print(f"{'Epsilon':>10} | {'Safe':>10} | {'Total':>10} | {'Safe %':>12} | {'Skipped':>10}")
print("-"*65)
for eps_str, r in safety_results.items():
    print(f"{float(eps_str):>10.4f} | {r['verified']:>10} | {r['total']:>10} | {r['verified_pct']:>10.1f}% | {r['skipped']:>10}")

In [ ]:
# Visualization
import matplotlib.pyplot as plt

eps_values = [float(e) for e in robustness_results.keys()]
robustness_pct = [r['verified_pct'] for r in robustness_results.values()]
safety_pct = [r['verified_pct'] for r in safety_results.values()]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Robustness
ax1 = axes[0]
ax1.plot(eps_values, robustness_pct, 'b-o', linewidth=2, markersize=8)
ax1.axhline(y=50, color='r', linestyle='--', label='50% threshold')
ax1.set_xlabel('Perturbation (ε)', fontsize=12)
ax1.set_ylabel('Verified (%)', fontsize=12)
ax1.set_title('Property 1: Local Robustness', fontsize=14)
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_ylim([0, 105])

# Plot 2: Safety
ax2 = axes[1]
colors = ['green' if p == 100 else ('orange' if p > 50 else 'red') for p in safety_pct]
ax2.bar(range(len(eps_values)), safety_pct, color=colors, alpha=0.7)
ax2.set_xticks(range(len(eps_values)))
ax2.set_xticklabels([f'{e:.3f}' for e in eps_values])
ax2.axhline(y=100, color='green', linestyle='-', linewidth=2, label='Safety verified')
ax2.set_xlabel('Perturbation (ε)', fontsize=12)
ax2.set_ylabel('Safe Samples (%)', fontsize=12)
ax2.set_title('Property 2: Safety (CRITICAL → never NON_CRITICAL)', fontsize=14)
ax2.legend()
ax2.grid(True, alpha=0.3, axis='y')
ax2.set_ylim([0, 105])

plt.tight_layout()
plt.savefig('verification_results.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Save results locally
final_results = {
    "timestamp": datetime.now().isoformat(),
    "model": {
        "type": "PointNetForVerification",
        "n_points": N_POINTS,
        "in_channels": IN_CHANNELS,
        "num_classes": NUM_CLASSES,
        "use_tnet": True,
        "feature_transform": True,
        "parameters": n_params,
        "test_accuracy": best_acc,
    },
    "verification_method": "auto_LiRPA (IBP+backward)",
    "n_verify_samples": N_VERIFY_SAMPLES,
    "property1_robustness": robustness_results,
    "property2_safety": safety_results,
}

with open('verification_results.json', 'w') as f:
    json.dump(final_results, f, indent=2)

print("Results saved locally:")
print("  - pointnet_best.pth (model checkpoint)")
print("  - verification_results.json")
print("  - verification_results.png")
print("  - training_curves.png")
print("\nRun the next cell to download all files to your computer.")

In [ ]:
# Download all files to your computer
from google.colab import files

print("Downloading files...")
files.download('pointnet_best.pth')
files.download('verification_results.json')
files.download('verification_results.png')
files.download('training_curves.png')
print("Done! Check your Downloads folder.")